In [2]:
import numpy as np
from collections import namedtuple

from pathlib import Path

In [51]:
Spec = namedtuple("Spec", ["mz", "int_"])

def range_sel_max(mz, int_, sels):
    def max_sel(_mz, _int_, _min, _max):
        v = _int_[(_mz > _min) & (mz < _max)]
        if v.size == 0:
            return 0
        else:
            return v.max()
    return np.asarray([max_sel(mz, int_, min, max) for (min, max) in sels])

class Regressor:
    def __init__(self, chem1:tuple[np.ndarray, np.ndarray], chem2:tuple[np.ndarray, np.ndarray], sel:list[tuple[float, float]]) -> None:
        from copy import deepcopy
        self.sel = deepcopy(sel)
        self.before = range_sel_max(chem1[0], chem1[1], self.sel)
        self.after = range_sel_max(chem2[0], chem2[1], self.sel)
        self.before /= self.before.max()
        self.after /= self.after.max()

        self.coef = np.column_stack((self.before, self.after))

    def peak_sel(self, data:list[Spec]):
        peaks = []
        for spec in data:
            _peak_int = range_sel_max(spec.mz, spec.int_, self.sel)
            peaks.append(_peak_int)
        return np.stack(peaks, 0)

    def calc(self, B, n_jobs=None):
        from scipy.optimize import nnls
        import joblib
        @joblib.delayed
        def _nnls(A, b):
            return nnls(A, b)[0]
        results = joblib.Parallel(n_jobs=n_jobs)(
        [_nnls(self.coef, B[i, :]) for i in range(B.shape[1])])
        results = np.stack(results, 0)
        results /= results.sum(1, keepdims=True)
        return results        

In [52]:
def read_data(fpath):
    return np.loadtxt(fpath, unpack=True, dtype=float, skiprows=2)

In [53]:
range_sel = [(2007.4 + i, 2008.4 + i) for i in range(7)]
D_x, D_y = np.loadtxt(r"data\D.txt", unpack=True, dtype=float)
N_x, N_y = np.loadtxt(r"data\N.txt", unpack=True, dtype=float)
regressor = Regressor((D_x, D_y), (N_x, N_y), range_sel)

In [54]:
N1D9 = []
for f in Path(r"data\Asn2Asp18").iterdir():
    x, y = read_data(f)
    N1D9.append(Spec(x, y))
N9D1 = []
for f in Path(r"data\Asn18Asp2").iterdir():
    x, y = read_data(f)
    N9D1.append(Spec(x, y))

In [55]:
N1D9 = regressor.peak_sel(N1D9)
N9D1 = regressor.peak_sel(N9D1)

In [62]:
print("N1D9")
print("D%: {obj[0]:%} \t N%: {obj[1]:%}".format(obj=regressor.calc(N1D9).mean(0)))
print("N9D1")
print("D%: {obj[0]:%} \t N%: {obj[1]:%}".format(obj=regressor.calc(N9D1).mean(0)))

N1D9
D%: 85.881271% 	 N%: 14.118729%
N9D1
D%: 2.894096% 	 N%: 97.105904%
